In [1]:
%load_ext autotime

In [2]:
from astropy.io import fits
import photutils
from photutils import Background2D, SigmaClip, MeanBackground, MMMBackground, \
    MedianBackground, SExtractorBackground, BkgIDWInterpolator, BkgZoomInterpolator

import numpy as np

from piaa import observation
from pocs.utils import current_time

time: 2.97 s


In [3]:
rgb_masks = np.load('/var/panoptes/rgb_masks.numpy')
camera_bias = 2048

time: 107 ms


In [4]:
image_dirs = {
    'crowded_field': '/var/panoptes/images/fields/Kic8462852/14d3bd/20170416T124816/',
    'new_moon': '/var/panoptes/images/fields/HatP20/14d3bd/20170128T053449/',
}

time: 993 µs


In [5]:
box_size = [
    (316, 434),
    (11, 12)
]

time: 5.37 ms


In [6]:
estimators = [MeanBackground(), MedianBackground(), MMMBackground()]
interpolators = [BkgZoomInterpolator()]
sigmas = [3, 4, 5]
iters = [5, 7, 10]

time: 2.2 ms


In [7]:
for name, image_dir in image_dirs.items():
    obs = observation.Observation(image_dir, verbose=True)

    data = fits.getdata(obs.files[1]) - camera_bias
    
    print("{}".format(name))
    for h, w in box_size:
        print("\tBox: {} {}".format(h, w))
        for estimator in estimators:
            print("\t{}".format(estimator))
            for interpolator in interpolators:
                print("\t{}".format(interpolator))
                for sigma in sigmas:
                    for iterations in iters:
                        print("\tSigma: {} Iters: {}".format(sigma, iterations))
                        start_time = current_time()
                        for color, mask in zip(['R', 'G', 'B'], rgb_masks):
                            sigma_clip = SigmaClip(sigma=sigma, iters=iterations)
                            bkg = Background2D(data, (h, w), filter_size=(3, 3),
                                           sigma_clip=sigma_clip, bkg_estimator=estimator, mask=~mask,
                                           interpolator=interpolator)
                            end_time = current_time()

                            print("\t{} Background\t Value: {:.02f}\t RMS: {:.02f}".format(
                                color, bkg.background_median, bkg.background_rms_median))

                        print("\tTotal time: {:0.2f} seconds\n".format((end_time - start_time).sec))

********************************************************************************
Setting up Observation for analysis
new_moon
	Box: 316 434
	Sigma: 3 Iters: 5
	R Background	 Value: 47.30	 RMS: 10.77
	G Background	 Value: 72.05	 RMS: 12.10
	B Background	 Value: 45.10	 RMS: 11.02
	Total time: 59.85 seconds

	Sigma: 3 Iters: 7
	R Background	 Value: 47.27	 RMS: 10.73
	G Background	 Value: 71.81	 RMS: 11.91
	B Background	 Value: 45.05	 RMS: 10.89
	Total time: 72.04 seconds

	Sigma: 3 Iters: 10
	R Background	 Value: 47.27	 RMS: 10.73
	G Background	 Value: 71.76	 RMS: 11.91
	B Background	 Value: 45.05	 RMS: 10.87
	Total time: 88.36 seconds

	Sigma: 4 Iters: 5
	R Background	 Value: 47.94	 RMS: 11.91
	G Background	 Value: 73.33	 RMS: 13.89
	B Background	 Value: 45.81	 RMS: 12.42
	Total time: 59.85 seconds

	Sigma: 4 Iters: 7
	R Background	 Value: 47.89	 RMS: 11.82
	G Background	 Value: 73.14	 RMS: 13.62
	B Background	 Value: 45.74	 RMS: 12.23
	Total time: 71.89 seconds

	Sigma: 4 Iters: 10
	R B